In [ ]:
#%pip install azure-ai-projects

## Example of using three Azure AI agents with AUTOGEN as the orchestrator

In [ ]:

from PIL import Image
from pathlib import Path
from IPython.display import display, HTML
import os
import sys
import time


from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

from azure.ai.agents import AgentsClient 
from azure.ai.agents.models import CodeInterpreterTool, BingGroundingTool
import os
import yaml

from dotenv import load_dotenv
load_dotenv()

from azure.identity import DefaultAzureCredential

In [ ]:
with open("model_config.yaml", "r") as f:
        model_config = yaml.safe_load(f)
model_client = AzureOpenAIChatCompletionClient.load_component(model_config)

In [ ]:
endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

In [ ]:
project_client = AgentsClient(
    credential=DefaultAzureCredential(),
    endpoint=endpoint
)

In [ ]:
# [START create_agent_with_bing_grounding_tool]
conn_id = os.getenv("AZURE_BING_CONNECTION_ID")

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

# define the web_ai_agent, with BING grounding

In [ ]:
async def web_ai_agent(query: str) -> str:
    print()
    print("This is Bing for Azure AI Agent Service .......")
    print()
    bing = BingGroundingTool(connection_id=conn_id)
    # with project_client:
    agent = project_client.create_agent(
            model="gpt-4.1-mini",
            name="my-web-agent",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.messages.create(
            thread_id=thread.id,
            role="user",
            content=query,
    )
    print(f"SMS: {message}")

    # Create and process agent run in thread with tools
    run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    #messages = project_client.messages.list(thread_id=thread.id)
    #print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])

    messages_paged = project_client.messages.list(thread_id=thread.id)
    messages = list(messages_paged)
    if messages:
        print("Messages:" + messages[0]["content"][0]["text"]["value"])
        return messages[0]["content"][0]["text"]["value"]
    else:
        print("No messages found.")
        return ""

        # project_client.close()

    #return messages["data"][0]["content"][0]["text"]["value"]



# Define the blog writer agent with Code Interpreter tool

In [21]:
from IPython.display import FileLink

async def save_blog_agent(blog_content: str) -> str:
    print()
    print("This is Code Interpreter for Azure AI Agent Service .......")

    code_interpreter = CodeInterpreterTool()
    agent = project_client.create_agent(
        model="gpt-4.1-mini",
        name="my-coder-agent",
        instructions="You are helpful agent",
        tools=code_interpreter.definitions,
    )

    thread = project_client.threads.create()

    message = project_client.messages.create(
        thread_id=thread.id,
        role="user",
        content=f"""
            You are my Python programming assistant. Generate code, save the following blog content, and execute it according to the requirements:

            {blog_content}

            1. Save blog content to blog-Agentic_AI_Blog.md
            2. Give me the download link for this file.
        """,
    )

    run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    messages_paged = project_client.messages.list(thread_id=thread.id)
    messages = list(messages_paged)
    if messages:
        print("Messages:" + messages[0]["content"][0]["text"]["value"])
    else:
        print("No messages found.")

    # Save the file to a notebook-accessible directory
    file_name = f"AI_Blog_{time.strftime('%Y%m%d%H%M%S')}.md"
    target_dir = "./blog"
    os.makedirs(target_dir, exist_ok=True)
    file_path = os.path.join(target_dir, file_name)
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(blog_content)
    print(f"File saved: {file_path}")

    # Display a download link in the notebook
    display(FileLink(file_path, result_html_prefix="Download your blog: "))

    project_client.delete_agent(agent.id)
    print("Deleted agent")

    return "Saved"

## Define the three agents

In [22]:
bing_search_agent = AssistantAgent(
    name="bing_search_agent",
    model_client=model_client,
    tools=[web_ai_agent],
    system_message="You are a search expert, help me use tools to find relevant knowledge",
)

save_blog_content_agent = AssistantAgent(
    name="save_blog_content_agent",
    model_client=model_client,
    tools=[save_blog_agent],
    system_message="""
        Save blog content. Respond with 'Saved' to when the blog is saved.
    """
)

write_agent = AssistantAgent(
    name="write_agent",
    model_client=model_client,
    system_message="""
        You are a blog writer, please help me write a blog based on bing search content."
    """
)


In [23]:
text_termination = TextMentionTermination("Saved")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(10)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

# Define the AUTOGEN team of Azure AI agents

In [24]:

reflection_team = RoundRobinGroupChat([bing_search_agent, write_agent,save_blog_content_agent], termination_condition=termination)

## start the team chat

In [25]:
await Console(
    reflection_team.run_stream(task="""

                    I am writing one blog about machine learning. 
                    Search for the following 4 questions and write a blog based on the search results ,
                    save it, and give me the download this file link
                    
                    1. What is Machine Learning?
                    2. The difference between AI and ML
                    3. The history of Machine Learning
                    4. The future of Machine Learning
                    
                    
    """)
)  # Stream the messages to the console.

---------- TextMessage (user) ----------


                    I am writing one blog about machine learning. 
                    Search for the following 4 questions and write a blog based on the search results ,
                    save it, and give me the download this file link

                    1. What is Machine Learning?
                    2. The difference between AI and ML
                    3. The history of Machine Learning
                    4. The future of Machine Learning


    

This is Bing for Azure AI Agent Service .......

Created agent, ID: asst_Ma65ZInAxfohdbT99059ebVH
Created thread, ID: thread_EWtWQiSR5yXEQLT6ThslvWMi
SMS: {'id': 'msg_k5aawufvAivKaMqyZuwNttm4', 'object': 'thread.message', 'created_at': 1750260974, 'assistant_id': None, 'thread_id': 'thread_EWtWQiSR5yXEQLT6ThslvWMi', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What is Machine Learning?', 'annotations': []}}], 'attachments': [], 'metadata': {}}
Run finishe

/Users/arturoquiroga/GITHUB/AZURE AI AGENT SERVICE/1_Azure Agent Service/NOTEBOOKS/AUTOGEN/blog/AI_Blog_20250618113716.md

Deleted agent
---------- ToolCallRequestEvent (save_blog_content_agent) ----------
[FunctionCall(id='call_R2M7MG9ynQvQagCotEW95Gjh', arguments='{"blog_content":"# Understanding Machine Learning: Past, Present, and Future\\n\\nMachine learning (ML) is one of the most fascinating and impactful areas in technology today. From powering voice assistants to enabling self-driving cars, machine learning is transforming how we interact with the world. In this blog, we’ll dive into what machine learning is, how it differs from artificial intelligence (AI), its rich history, and what the future may hold for this transformative field.\\n\\n---\\n\\n## What is Machine Learning?\\n\\nMachine learning is a branch of artificial intelligence that focuses on enabling computers to learn from data rather than follow explicit instructions. Instead of programming fixed rules to perform tasks, ML algorithms identify patterns and relationships within data to make predictions or decisions on new, unseen inform

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='\n\n                    I am writing one blog about machine learning. \n                    Search for the following 4 questions and write a blog based on the search results ,\n                    save it, and give me the download this file link\n\n                    1. What is Machine Learning?\n                    2. The difference between AI and ML\n                    3. The history of Machine Learning\n                    4. The future of Machine Learning\n\n\n    ', type='TextMessage'), ToolCallRequestEvent(source='bing_search_agent', models_usage=RequestUsage(prompt_tokens=144, completion_tokens=94), metadata={}, content=[FunctionCall(id='call_CQRmC6iQBdM7P1PwP3irr2Yn', arguments='{"query": "What is Machine Learning?"}', name='web_ai_agent'), FunctionCall(id='call_PVHu5bAhb4bHSK6R5DjVJrLx', arguments='{"query": "Difference between Artificial Intelligence and Machine Learning"}', name='web_a

In [26]:
await Console(
    reflection_team.run_stream(task="""

                    I am writing one blog about agentic AI. 
                    Search for the following 4 questions and write a blog based on the search results ,
                    save it, and give me the download this file link

                    1. What is Agentic AI?
                    2. The difference between AI and Agentic AI
                    3. The history of Agentic AI
                    4. The future of Agentic AI


    """)
)  # Stream the messages to the console.

---------- TextMessage (user) ----------


                    I am writing one blog about agentic AI. 
                    Search for the following 4 questions and write a blog based on the search results ,
                    save it, and give me the download this file link

                    1. What is Agentic AI?
                    2. The difference between AI and Agentic AI
                    3. The history of Agentic AI
                    4. The future of Agentic AI


    

This is Bing for Azure AI Agent Service .......

Created agent, ID: asst_4h2tjdbH0BrEEetjcBSutMnA
Created thread, ID: thread_SxKelTDJY4gZr2mUBypmYsEL
SMS: {'id': 'msg_BGUTA7mvgAdmtQm78iN3sxH3', 'object': 'thread.message', 'created_at': 1750261080, 'assistant_id': None, 'thread_id': 'thread_SxKelTDJY4gZr2mUBypmYsEL', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What is Agentic AI?', 'annotations': []}}], 'attachments': [], 'metadata': {}}
Run finished with status: RunStat

/Users/arturoquiroga/GITHUB/AZURE AI AGENT SERVICE/1_Azure Agent Service/NOTEBOOKS/AUTOGEN/blog/AI_Blog_20250618113908.md

Deleted agent
---------- ToolCallRequestEvent (save_blog_content_agent) ----------
[FunctionCall(id='call_4wmbPqgmTEs7HpCeW4GpdRtC', arguments='{"blog_content":"# Agentic AI: The Next Generation of Autonomous Intelligence\\n\\nArtificial intelligence has evolved rapidly over the decades, and one of its most exciting frontiers today is **Agentic AI**\\u000d\\u000asystems that not only perform tasks but act independently as autonomous agents. In this blog, we explore what Agentic AI is, how it differs from traditional AI, its historical development, and what the future holds for this transformative technology.\\n\\n---\\n\\n## What is Agentic AI?\\n\\nAgentic AI refers to artificial intelligence systems designed to operate autonomously as agents that perceive their environment, set goals, make decisions, and take actions with minimal or no human intervention. Unlike traditional AI models\\u000d\\u000awhich are largely reactive and often require explicit human prompts\\u000d\\u000aagentic

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='\n\n                    I am writing one blog about agentic AI. \n                    Search for the following 4 questions and write a blog based on the search results ,\n                    save it, and give me the download this file link\n\n                    1. What is Agentic AI?\n                    2. The difference between AI and Agentic AI\n                    3. The history of Agentic AI\n                    4. The future of Agentic AI\n\n\n    ', type='TextMessage'), ToolCallRequestEvent(source='bing_search_agent', models_usage=RequestUsage(prompt_tokens=2718, completion_tokens=97), metadata={}, content=[FunctionCall(id='call_gbizpUavyknnvNlzJe5niIff', arguments='{"query": "What is Agentic AI?"}', name='web_ai_agent'), FunctionCall(id='call_KgVTnPe1ZWKs79VYyseWMk6G', arguments='{"query": "Difference between AI and Agentic AI"}', name='web_ai_agent'), FunctionCall(id='call_ptD49vw22n3aJ53

In [ ]:


await Console(
    reflection_team.run_stream(task="""

                   I am writing one blog about cybersecurity.
                    Search for the following 4 questions and write a blog based on the search results,
                    save it, and give me the download this file link

                    - What is Cybersecurity?
                    - Differences between cybersecurity and information security
                    - The evolution of cybersecurity practices over time
                    - The future trends and threats in cybersecurity

    """)
)  # Stream the messages to the console.